In [1]:
import time
import datetime
import pandas as pd
import chromedriver_binary
from tqdm import tqdm
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import numpy as np

pd.set_option("display.max_rows",
              None, 
              "display.max_columns",
              None)

In [2]:
import chromedriver_autoinstaller as chromedriver
chromedriver.install()

'/opt/homebrew/lib/python3.9/site-packages/chromedriver_autoinstaller/111/chromedriver'

In [3]:
browser = webdriver.Chrome()

In [4]:
df = pd.read_excel('./data/hl_cases_sample.xlsx', index_col=False)

In [5]:
def scrape(hl_df):
    
    cases_info_list = []

    for i in tqdm(range(0, len(hl_df))):

        browser.get("https://wcca.wicourts.gov/advanced.html")

        time.sleep(3)

        county_name = browser.find_element(By.XPATH, "//*[@id='react-select-2--value']/div[2]/input")
        county_name.send_keys(str(hl_df['County name'][i]) + Keys.ENTER)

        case_number = browser.find_element(By.NAME, "caseNo")
        case_number.send_keys(str(hl_df['Case number'][i]) + Keys.ENTER)

        search = browser.find_element(By.NAME, "search")
        search.click()

        time.sleep(16)

        cs_n = str(hl_df['Case number'][i])
        ct_num = str(hl_df['County name'][i])

        try: 
            cs_filling_date = browser.find_element(By.XPATH, "//*[@id='summary']/div[1]/dl[1]/dd").text
        except:
            cs_filling_date = np.nan
            print('Does not contain cs_filling_date')
            pass

        try: 
            cs_case_type = browser.find_element(By.XPATH, "//*[@id='summary']/div[1]/dl[2]/dd").text
        except:
            cs_case_type = np.nan
            print('Does not contain cs_case_type')
            pass
        
        try: 
            cs_case_status = browser.find_element(By.XPATH, "//*[@id='summary']/div[1]/dl[3]/dd").text
        except:
            cs_case_status = np.nan
            print('Does not contain cs_case_status')
            pass

        try: 
            for j in range(2,10):
                if 'Plaintiff' in browser.find_element(By.XPATH, "//*[@id='parties']/div["+str(j)+"]/div/h5").text:
                    pt_plaintiff = browser.find_element(By.XPATH, "//*[@id='parties']/div["+str(j)+"]/div/h5").text
                    break
        except:
            pt_plaintiff = np.nan
            print('Does not contain pt_plaintiff')
            pass

        try: 
            for j in range(2,10):
                if 'Plaintiff' in browser.find_element(By.XPATH, "//*[@id='parties']/div["+str(j)+"]/div/h5").text:
                    pt_plaintiff_address = browser.find_element(By.XPATH, "//*[@id='parties']/div["+str(j)+"]/div/div/div[2]/div/dl/dd").text
                    break
        except:
            pt_plaintiff_address = np.nan
            print('Does not contain pt_plaintiff_address')
            pass

        try: 
            for v in range(2,10):
                if 'Injured' in browser.find_element(By.XPATH, "//*[@id='parties']/div["+str(v)+"]/div/h5").text:
                    pt_inj_party = browser.find_element(By.XPATH, "//*[@id='parties']/div["+str(v)+"]/div/h5").text
                    break
        except:
            pt_inj_party = np.nan
            print('Does not contain pt_inj_party')
            pass

        try: 
            for j in range(2,10):
                if 'Injured' in browser.find_element(By.XPATH, "//*[@id='parties']/div["+str(j)+"]/div/h5").text:
                    pt_inj_party_address = browser.find_element(By.XPATH, "//*[@id='parties']/div["+str(j)+"]/div/div/div[2]/div/dl/dd").text
                    break
        except:
            pt_inj_party_address = np.nan
            print('Does not contain pt_inj_party_address')
            pass

        try: 
            cjd_county = browser.find_element(By.XPATH, "//*[@id='civilJdgmts']/div/div/div[1]/dl[1]/dd").text
        except:
            cjd_county = ct_num
            print('County name: ' + hl_df['County name'][i])
            pass

        try: 
            cjd_case_number = browser.find_element(By.XPATH, "//*[@id='civilJdgmts']/div/div/div[1]/dl[2]/dd").text
        except:
            cjd_case_number = cs_n
            print('Case number: ' + str(hl_df['Case number'][i]))
            pass

        try: 
            cjd_total_amount = browser.find_element(By.XPATH, "//*[@id='civilJdgmts']/div/div/div[1]/dl[5]/dd").text
        except:
            cjd_total_amount = np.nan
            print('Does not contain cjd_total_amount')
            pass

        try: 
            cjd_satisfaction = browser.find_element(By.XPATH, "//*[@id='civilJdgmts']/div/div/div[2]/dl[1]/dd").text
        except:
            cjd_satisfaction = np.nan
            print('Does not contain cjd_satisfaction')
            pass

        try: 
            cjd_judgment_status = browser.find_element(By.XPATH, "//*[@id='civilJdgmts']/div/div/div[2]/dl[2]/dd").text
        except:
            cjd_judgment_status = np.nan
            print('Does not contain cjd_judgment_status')
            pass

        try: 
            for x in range(1,5):
                if 'Judgment parties' in browser.find_element(By.XPATH, "//*[@id='civilJdgmts']/div/div/table["+str(x)+"]/caption").text:
                    for w in range(1,10):
                        if 'Creditor' in browser.find_element(By.XPATH, "//*[@id='civilJdgmts']/div/div/table["+str(x)+"]/tbody/tr["+str(w)+"]/td[1]").text:
                            cjd_creditor_name = browser.find_element(By.XPATH, "//*[@id='civilJdgmts']/div/div/table["+str(x)+"]/tbody/tr["+str(w)+"]/td[2]").text
                            cjd_creditor_address = browser.find_element(By.XPATH, "//*[@id='civilJdgmts']/div/div/table["+str(x)+"]/tbody/tr["+str(w)+"]/td[5]").text
                            cjd_creditor_attorney_name = browser.find_element(By.XPATH, "//*[@id='civilJdgmts']/div/div/table["+str(x)+"]/tbody/tr["+str(w)+"]/td[6]").text
                            break
                    break
        except:
            cjd_creditor_name = np.nan
            cjd_creditor_address = np.nan
            cjd_creditor_attorney_name = np.nan
            print('Does not contain cjd_creditor_name | cjd_creditor_address | cjd_creditor_attorney_name')
            pass

        cases_info_list.append(
            {
                'cs_filling_date': cs_filling_date,
                'cs_case_type': cs_case_type,
                'cs_case_status': cs_case_status,
                'pt_plaintiff': pt_plaintiff,
                'pt_plaintiff_address': pt_plaintiff_address,
                'pt_inj_party': pt_inj_party,
                'pt_inj_party_address': pt_inj_party_address,
                'cjd_county': cjd_county,
                'cjd_case_number': cjd_case_number,
                'cjd_total_amount': cjd_total_amount,
                'cjd_satisfaction': cjd_satisfaction,
                'cjd_judgment_status': cjd_judgment_status,
                'cjd_creditor_name': cjd_creditor_name,
                'cjd_creditor_address': cjd_creditor_address,
                'cjd_creditor_attorney_name': cjd_creditor_attorney_name,
            }
        )
    
    browser.close()

    case_page_df = pd.DataFrame(cases_info_list)

    return case_page_df

In [10]:
case_page_df = scrape(df[:10])

100%|██████████| 10/10 [03:19<00:00, 19.96s/it]


In [13]:
case_page_df.head()

,cs_filling_date,cs_case_type,cs_case_status,pt_plaintiff,pt_plaintiff_address,pt_inj_party,pt_inj_party_address,cjd_county,cjd_case_number,cjd_total_amount,cjd_satisfaction,cjd_judgment_status,cjd_creditor_name,cjd_creditor_address,cjd_creditor_attorney_name
0,01-16-2015,Hospital Lien,Filed Only,Plaintiff: Meriter Hospital,,"Injured party: Decono, Joseph D","125 Capitol View Ter Apt 4, Madison, WI 53713",Dane,2015HL000022,"$1,325.00",Full,Full satisfaction,Meriter Hospital,,
1,01-16-2015,Hospital Lien,Filed Only,Plaintiff: UWHCA,,"Injured party: Schmitzer, Steven","507 3rd Ave, Monroe, WI 53566-3736",Dane,2015HL000021,"$90,309.10",Full,Full satisfaction,UWHCA,,
2,01-16-2015,Hospital Lien,Filed Only,Plaintiff: UWHCA,,"Injured party: Breach, Holly","2503 Dyreson Rd, Mc Farland, WI 53558",Dane,2015HL000020,"$22,393.71",Full,Full satisfaction,UWHCA,,
3,01-16-2015,Hospital Lien,Filed Only,"Plaintiff: Theda Clark Medical Center, Inc.","130 Second Street, Neenah, WI 54956","Injured party: Shirriffs, Taylor G","2775 Beechnut Drive, Oshkosh, WI 54904",Winnebago,2015HL000002,"$59,624.45",Full,Full satisfaction,"Theda Clark Medical Center, Inc.","130 Second Street, Neenah, WI 54956","Erwin, Bridget M."
4,01-16-2015,Hospital Lien,Filed Only,Plaintiff: Theda Clark Medical Center,"130 Second St, Neenah, WI 54956","Injured party: Schroeder, Hunter R L","800 S Main St, Seymour, WI 54165",Outagamie,2015HL000002,"$26,269.70",Other,Release,Theda Clark Medical Center,"130 Second St, Neenah, WI 54956","Erwin, Bridget M."


In [12]:
case_page_df.to_excel('./data/hl_case_page_sample.xlsx', index=False)